In [2]:
import itertools

In [3]:
from pybtex.database import parse_file as pybtex_parse_file

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [5]:
%matplotlib inline

In [6]:
vispapers = pybtex_parse_file('groups-in-graphs.corinna-vehlow.com.bib')

In [40]:
G = nx.Graph()

In [41]:
from collections import Counter
all_keywords = {}
for name, entry in vispapers.entries.iteritems():
    keywords = entry.fields['keywords'].split(', ')
    for k in keywords:
        if ':' in k:
            k = k.split(':')
            if k[0] not in all_keywords:
                all_keywords[k[0]] = Counter()
            all_keywords[k[0]][k[1]] += 1
        else:
            if k not in all_keywords:
                all_keywords[k] = Counter()
            all_keywords[k]['_'] += 1
entry.fields['title']
entry.fields['keywords'].split(', ')

[u'paper_type:technique',
 u'group_type:vertex-based',
 u'graph_vis:node-link',
 u'group_overlap:overlapping_crisp',
 u'group_structure:flat',
 u'graph:generic',
 u'group_origin:categorical_attr',
 u'application:social_network',
 u'publication_channel:InfoVis_J',
 u'edge-group_vis:none',
 u'group_vis:juxtaposed',
 u'visual_attr:!',
 u'juxtaposed_vis:separate',
 u'superimposed_vis:!',
 u'embedded_vis:!',
 u'implicit_vis:!',
 u'evaluation:case_study',
 u'radial']

In [42]:
all_keywords

{u'2.5d': Counter({'_': 5}),
 u'3d': Counter({'_': 9}),
 u'aggregation': Counter({'_': 46}),
 u'application': Counter({u'biology': 51,
          u'computer_network': 15,
          u'document': 11,
          u'economy': 16,
          u'generic': 45,
          u'media': 5,
          u'social_network': 64,
          u'software_engineering': 34,
          u'sports': 4}),
 u'attribute-driven_layout': Counter({'_': 3}),
 u'circular_layout': Counter({'_': 22}),
 u'co-author': Counter({'_': 9}),
 u'color-optimization': Counter({'_': 5}),
 u'contour_filled': Counter({'_': 35}),
 u'edge-bundling': Counter({'_': 16}),
 u'edge-group_vis': Counter({u'embedded': 2,
          u'juxtaposed': 4,
          u'none': 219,
          u'superimposed': 1,
          u'visual_attr': 14}),
 u'embedded_vis': Counter({u'!': 191, u'hybrid': 5, u'node-link': 19}),
 u'energy_minimization_layout': Counter({'_': 4}),
 u'evaluation': Counter({u'algorithmic': 19,
          u'case_study': 88,
          u'comparison': 9,
 

In [43]:
for name, entry in vispapers.entries.iteritems():
    authors_this_paper = []
    this_paper = {
        'title': entry.fields['title'], 
        'year': entry.fields['year'], 
        'keywords': entry.fields['keywords'].split(', ')
    }
    for author in entry.persons['author']:
        #author_name = "{}, {}".format(author.get_part_as_text('last'), author.get_part_as_text('first'))
        author_name = "{} {}".format(author.get_part_as_text('first'), author.get_part_as_text('last'))
        author_name = author_name.replace('\\"', '')
        if G.has_node(author_name):
            G.node[author_name]['papers'].append(this_paper)
        else:
            G.add_node(author_name, attr_dict={'author_name': author_name, 'papers': [this_paper]})
        authors_this_paper.append(author_name)
    if len(authors_this_paper) > 1:
        for a1, a2 in itertools.combinations(authors_this_paper, 2):
            if G.has_edge(a1, a2):
                G[a1][a2]['weight'] += 1
            else:
                G.add_edge(a1, a2, weight=1)
        

In [44]:
components = list(nx.connected_components(G))

In [45]:
for a, b, data in sorted(G.edges(data=True), key=lambda x: x[2]['weight'], reverse=True):
    print('{a} | {b} | {w}'.format(a=a, b=b, w=data['weight']))

Daniel Weiskopf | Corinna Vehlow | 8
Michael Burch | Daniel Weiskopf | 7
Stephan Diehl | Fabian Beck | 6
Stephan Diehl | Michael Burch | 5
Walter Didimo | Giuseppe Liotta | 5
Daniel Archambault | David Auber | 5
Daniel Archambault | Tamara Munzner | 5
Tamara Munzner | David Auber | 5
Peter Eades | Qing-Wen Feng | 4
Romain Bourqui | David Auber | 4
Heorhiy Byelas | Alexandru Telea | 4
Tim Dwyer | Kim Marriott | 4
Chris Muelder | Kwan-Liu Ma | 4
Jean-Daniel Fekete | Nathalie Henry | 4
Stephen Kobourov | Yifan Hu | 4
Walter Didimo | Fabrizio Montecchiani | 4
Michael Burch | Fabian Beck | 3
Michael Burch | Corinna Vehlow | 3
Daniel Weiskopf | Fabian Beck | 3
Romain Bourqui | Fabien Jourdan | 3
Fabian Beck | Corinna Vehlow | 3
A. Pretorius | Jarke Wijk | 3
Marc Streit | Alexander Lex | 3
Marc Streit | Dieter Schmalstieg | 3
Alexander Lex | Dieter Schmalstieg | 3
Quang Nguyen | Mao Huang | 3
Yaniv Frishman | Ayellet Tal | 3
David Auber | Fabien Jourdan | 3
Ozan Ersoy | Alexandru Telea | 2
Na

In [46]:
import igraph

In [47]:
g2 = igraph.Graph.Adjacency((nx.to_numpy_matrix(G)).tolist())

In [48]:
g2.community_infomap()

In [49]:
# nx.write_pajek(G, 'coauthorship.net')

In [50]:
# # need to clean up the pajek file: only need, the number and name for the vertices
# with open('coauthorship.net', 'r') as f:
#     pjk_txt = f.readlines()
# this_line_is_vertex = False
# with open('coauthorship.net', 'w') as outf:
#     for line in pjk_txt:
#         if line[0] == '*':
#             if line[1].lower() == 'v':
#                 this_line_is_vertex = True
#             else:
#                 this_line_is_vertex = False
#         elif this_line_is_vertex:
#             line = line[:line.find(' 0.0')]
#             line = line + '\n'
#         outf.write(line)

In [51]:
with open('coauthorship.tree', 'r') as f:
    rows = []
    for line in f:
        if line[0] == '#':
            continue
        
        line = line.strip().split()
        author = line[2].strip('"')
        node = line[3]
        cl = line[0]
        flow = line[1]
        row = ( node, author, cl, flow )
        rows.append(row)

In [52]:
df = pd.DataFrame(rows, columns='node author cl flow'.split())

In [53]:
df = pd.read_csv('coauthorship.tree', skiprows=2, quotechar='"', delimiter=' ', names='cl flow author node'.split())

In [54]:
df['cl_bottom'] = df.cl.apply(lambda x: ':'.join(x.split(':')[:-1]))
df['cl_top'] = df.cl.apply(lambda x: x.split(':')[0])

In [55]:
df.set_index('author', inplace=True)

In [56]:
df

,cl,flow,node,cl_bottom,cl_top
author,,,,,
Daniel Weiskopf,1:1:1:1,0.018467,58,1:1:1,1
Corinna Vehlow,1:1:1:2,0.014774,193,1:1:1,1
Michael Burch,1:1:1:3,0.011542,34,1:1:1,1
Stephan Diehl,1:1:1:4,0.010157,32,1:1:1,1
Fabian Beck,1:1:1:5,0.009234,125,1:1:1,1
Peter Fritz,1:1:1:6,0.001385,38,1:1:1,1
Hansj{{o}}rg Schmauder,1:1:1:7,0.001385,75,1:1:1,1
Patrick Auw{a}rter,1:1:1:8,0.001385,113,1:1:1,1
Marcel Hlawatsch,1:1:1:9,0.000923,84,1:1:1,1


In [57]:
from collections import Counter
c = Counter()
for comp in components:
    c[len(comp)] += 1

In [58]:
c

Counter({1: 5,
         2: 24,
         3: 13,
         4: 3,
         5: 5,
         6: 2,
         7: 6,
         9: 4,
         10: 2,
         11: 1,
         12: 1,
         14: 1,
         23: 1,
         31: 1,
         127: 1})

In [59]:
nx.set_node_attributes(G, 'cl_bottom', df.cl_bottom.to_dict())
nx.set_node_attributes(G, 'cl_top', df.cl_top.to_dict())
nx.set_node_attributes(G, 'flow', df.flow.to_dict())

In [60]:
G.nodes(data=True)

[('Daniel Jackson',
  {'author_name': 'Daniel Jackson',
   'cl_bottom': '27',
   'cl_top': '27',
   'flow': 0.0013850399999999999,
   'papers': [{'keywords': [u'paper_type:application',
      u'group_type:vertex-based',
      u'graph_vis:matrix',
      u'group_overlap:disjoint',
      u'group_structure:hierarchical',
      u'edge-group_vis:none',
      u'group_vis:juxtaposed',
      u'visual_attr:!',
      u'juxtaposed_vis:attached',
      u'superimposed_vis:!',
      u'embedded_vis:!',
      u'implicit_vis:1D_layout',
      u'graph:directed',
      u'group_origin:generic',
      u'evaluation:case_study',
      u'application:software_engineering',
      u'publication_channel:oopsla',
      u'aggregation'],
     'title': u'Using Dependency Models to Manage Complex Software Architecture',
     'year': u'2005'}]}),
 ('Michael Kaufmann',
  {'author_name': 'Michael Kaufmann',
   'cl_bottom': '13',
   'cl_top': '13',
   'flow': 0.0027700799999999999,
   'papers': [{'keywords': [u'paper_type:

In [61]:
G = nx.relabel_nodes(G, df.node.to_dict())

In [62]:
G.nodes()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [63]:
i = 0
G.graph['connected_components'] = {}
for component in nx.connected_components(G):
    G.graph['connected_components'][i] = list(component)
    for n in component:
        G.node[n]['component'] = i
    i += 1

In [64]:
from networkx.readwrite import json_graph
import json

In [65]:
json_data = json_graph.node_link_data(G)
with open('coauthorship.json', 'w') as outf:
    json.dump(json_data, outf)

In [40]:
with open('coauthorship.json', 'r') as f:
    j = json.load(f)

In [29]:
ctypes = Counter()
for node in j['nodes']:
    ctypes[len(node)] += 1

In [30]:
ctypes

Counter({4: 457})

In [31]:
G.nodes(data=True)

[(1, {'author_name': 'Wu, Jie', 'cl_bottom': '5:1', 'cl_top': '5'}),
 (2, {'author_name': 'Smith, Adam', 'cl_bottom': '20', 'cl_top': '20'}),
 (3, {'author_name': 'Nakazawa, Rina', 'cl_bottom': '2:1:2:2', 'cl_top': '2'}),
 (4, {'author_name': 'Mader, Martin', 'cl_bottom': '12', 'cl_top': '12'}),
 (5, {'author_name': 'Sangal, Neeraj', 'cl_bottom': '27', 'cl_top': '27'}),
 (6, {'author_name': 'Elmqvist, Niklas', 'cl_bottom': '4:1', 'cl_top': '4'}),
 (7, {'author_name': 'Philippi, Stephan', 'cl_bottom': '8', 'cl_top': '8'}),
 (8, {'author_name': 'Vernik, Rudi', 'cl_bottom': '30', 'cl_top': '30'}),
 (9,
  {'author_name': 'Goesmann, Alexander', 'cl_bottom': '21', 'cl_top': '21'}),
 (10, {'author_name': 'Tanaka, Jiro', 'cl_bottom': '22:1', 'cl_top': '22'}),
 (11, {'author_name': 'Mary, Patrick', 'cl_bottom': '2:1:1:1', 'cl_top': '2'}),
 (12, {'author_name': 'Nguyen, QuanHoang', 'cl_bottom': '12', 'cl_top': '12'}),
 (13, {'author_name': 'Kramer, Andrei', 'cl_bottom': '1:1:2', 'cl_top': '1'}),

In [32]:
deg = pd.DataFrame.from_dict(G.degree(), orient='index').rename(columns={0: 'degree'})

In [33]:
deg.degree.value_counts()

2     91
4     75
3     70
1     60
6     50
5     37
8     29
7     13
11     7
9      5
0      5
10     4
12     4
13     2
20     1
14     1
16     1
18     1
22     1
Name: degree, dtype: int64

In [48]:
df['cl_depth'] = df.cl_bottom.apply(lambda x: len(x.split(':')))

In [53]:
df[df.cl_depth>1].cl_bottom.value_counts().shape

(41,)